In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataprep-rslex 2.6.3 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-dataprep-rslex~=2.3.0dev0'), {'azureml-dataprep'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = az

In [22]:
from obs.drift.drift_analysis_job import execute_drift_detect_job
from datetime import datetime
from azure.ai.ml.constants import TimeZone
from dateutil import tz

schedule_start_time = datetime.now(tz=tz.gettz("PACIFIC STANDARD TIME"))
cron_schedule = CronSchedule(
    expression="*/10 * * * *",
    start_time=schedule_start_time,
    time_zone=TimeZone.PACIFIC_STANDARD_TIME,
    status=ScheduleStatus.ENABLED,
)

ml_client, job_name = execute_drift_detect_job(subscription_id="0e9bace8-7a81-4922-83b5-d995ff706507",resource_group="azureml",workspace="ws01ent", compute_name ='DS11', experiment_name= "drift-analysis-job", base_table_name ="ISDWeather", 
target_table_name ="ISDWeather", base_dt_from ="2013-05-13", base_dt_to= "2014-06-13",target_dt_from="2013-04-13", target_dt_to="2014-05-13", bin="7d", limit=3000000, cron_schedule =None,user_defined_module_file="drift_analysis_user.py", user_defined_conda_file="conda.yml")

Uploading .tmp (0.01 MBs): 100%|##########| 12970/12970 [00:00<00:00, 133923.16it/s]




PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '.tmp\\conda.yml'

In [25]:
def cancel_job(ml_client, job_name):
    from azure.ai.ml.entities import (
        CronSchedule,
        RecurrenceSchedule,
        RecurrencePattern,
        ScheduleStatus,
    )


    existing_job = ml_client.jobs.get(name=job_name)
    # change schedule status to disable
    existing_job.schedule.status = ScheduleStatus.DISABLED
    ml_client.jobs.create_or_update(existing_job, experiment_name=existing_job.experiment_name)
cancel_job(ml_client, job_name)


In [ ]:
job_name

In [26]:
for job in ml_client.jobs.list():
    if job.display_name == "schedule_pipeline" and job.schedule.status!="disabled":
        cancel_job(ml_client, job.name)


additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_properties is not a known attribute of class <class 'azure.ai.ml.entities._job.distribution.MpiDistribution'> and will be ignored
additional_pr

KeyboardInterrupt: 

In [5]:
from aml_obs.query import RT_Visualization
# streaming_table_name="streaming_test"

# rt_viz =RT_Visualization(streaming_table_name,ws)
# rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Anomaly Detection

In [ ]:
from aml_obs.query import KustoQuery
query = KustoQuery(streaming_table_name, ws)

query.anomaly_detection(min_t="4/26/2022 10:08:45", max_t="4/23/2022 10:10:00", step="0.5s", metric="metric1", agg="avg", ts_col="timestamp", groupby="lr", sensitivity=0.1, filter="0.001")


### Drift Query

In [1]:

from aml_obs.drift import Drift_Analysis

In [3]:
from aml_obs.drift import Drift_Analysis
drift_analysis =Drift_Analysis(ws)
#test1 1m 1m28 vs 2m26
#test2 10m 1459 vs 10m59


In [4]:
drift_analysis.sample_data_points("StormEvents", col_name="EventId", start_date = "01/01/2007", horizon=30 )

,EventId
0,13208
1,23358
2,23357
3,9494
4,9488
...,...
3261,9963
3262,12466
3263,9969
3264,11485


In [5]:
df_output = drift_analysis.analyze_drift(limit=10000,base_table_name = 'ISDWeather',target_table_name='ISDWeather', base_dt_from='2013-04-13', base_dt_to='2014-05-13', target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')


In [ ]:
feature_distribution = drift_analysis.get_features_distributions(target_table_name='ISDWeather',target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')

In [9]:
from aml_obs.drift import execute_drift_detect_job
execute_drift_detect_job(subscription_id="0e9bace8-7a81-4922-83b5-d995ff706507",resource_group="azureml",workspace="ws01ent", compute_name ='DS11', experiment_name= "drift-analysis-job", base_table_name ="ISDWeather", 
target_table_name ="ISDWeather", base_dt_from ="2013-05-13", base_dt_to= "2014-06-13",target_dt_from="2013-04-13", target_dt_to="2014-05-13", bin="7d", limit=3000000)

Uploading .tmp (0.0 MBs): 100%|##########| 3480/3480 [00:00<00:00, 31304.40it/s]




In [14]:
# #Option to set parameters to a custom ADX cluster
# from azure.mgmt.kusto import KustoManagementClient
# from azure.mgmt.kusto.models import Cluster, AzureSku
# from azure.common.credentials import ServicePrincipalCredentials
# from azure.mgmt.kusto import KustoManagementClient
# from azure.mgmt.kusto.models import Cluster, AzureSku,LanguageExtension, LanguageExtensionsList
# from azure.mgmt.kusto.models import ReadWriteDatabase
# from azure.mgmt.kusto.models import DatabasePrincipalAssignment
# from azure.identity import ClientSecretCredential 


# tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# #Application ID
# client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
# #Client Secret, set it at your WS' keyvault with key name same as your client_id
# client_secret = kv.get_secret(client_id)
# subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

# cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster
# # kv.set_secret(KV_SP_ID,client_id)
# # kv.set_secret(KV_SP_KEY,client_secret)
# # kv.set_secret(KV_ADX_DB,"db01")
# # kv.set_secret(KV_ADX_URI,cluster_uri)
# # kv.set_secret(KV_TENANT_ID,tenant_id)

# credentials = ClientSecretCredential(
#     client_id=client_id,
#     client_secret=client_secret,
#     tenant_id=tenant_id
# )
# cluster_name = "adx02"
# resource_group_name = "databackend"
# kusto_management_client = KustoManagementClient(credentials, subscription_id)

# cluster_operations = kusto_management_client.clusters





In [22]:
# poller = cluster_operations.begin_add_language_extensions(resource_group_name, cluster_name, LanguageExtensionsList(value=[LanguageExtension(language_extension_name="PYTHON")]))
# poller.wait()